In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from numpy import mean
from matplotlib_venn import venn2
from matplotlib.pyplot import figure

def save_predictions(data, file_name):
    df = pd.read_csv('./data/gender_submission.csv')
    df['Survived'] = data
    return df.to_csv(file_name, index=False)

def correlation_graph(data):
    corr = data.corr()
    mask = np.triu(np.ones_like(corr, dtype=np.bool)) # Generate a mask for the upper triangle
    cmap = sns.diverging_palette(290, 10, as_cmap=True) # Generate a custom diverging colormap
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.6, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .9}) # Draw the heatmap with the mask and correct aspect ratio
